In [5]:
import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm import tqdm



# lessons learned:
# 1. the problem should feasible, convex, for any given parameters
# 2. Make sure the there is no product of two parameters that to be inferred



# ----------------- 1. Define the MPC problem -----------------



from cvxpylayers.torch import CvxpyLayer
torch.set_default_dtype(torch.double)
torch.set_default_device('cpu')

# define the nominal dynamics
n = 4 # state dimension
m = 1 # control dimension
dt = 0.1 # time integrator
A_np = np.array([[1., dt],[0, 1.]]) # dynamics of the ego car
A = np.kron(np.eye(2), A_np) # dynamics of the two cars
B1 = np.array([0,dt,0,0]).reshape(-1,1) # control of the ego car
B2 = np.array([0,0,0,dt]).reshape(-1,1) # control of the other car

A_torch = torch.tensor(A)
B1_torch = torch.tensor(B1)
B2_torch = torch.tensor(B2)


# define the ground truth parameters of the cost function
# parameters:   
#               weight_on_velocity_cost, 
#               weight_on_acceleration_cost, 
#               desired_distance, 
#               desired_velocity,
# ground_truth_parameters = torch.tensor([1.0, 1.0, 0.5, 1.0])
# ground_truth_parameters_np = ground_truth_parameters.cpu().numpy()
control_limit = 1.0 # this is the constraint on the control
T = 5 # look-ahead horizon


# define the dynamics and cost function
def dynamics_torch(xt, ut, acceleration_of_other_car = torch.tensor([0.])):
    return A_torch @ xt + B1_torch @ ut + B2_torch @ acceleration_of_other_car

def dynamics_np(xt, ut, acceleration_of_other_car = np.array([0.])):
    return A @ xt + B1 @ ut + B2 @ acceleration_of_other_car




def construct_differentiable_mpc_problem(desired_speed):
    # belows are the parameters of the MPC problem
    x = cp.Parameter(n)
    inferred_parameters = cp.Parameter(4, nonneg=True) # this is the parameters of the problem
    acceleration_of_other_car = cp.Parameter(1)
    # belows are the decision variables of the MPC problem
    states = [cp.Variable(n) for _ in range(T+1)]
    controls = [cp.Variable(m) for _ in range(T)]
    # initial constraints
    constraints = [states[0] == x, 
        cp.norm(controls[0], 'inf') <= control_limit,
        states[0][0] <= states[0][2] - inferred_parameters[2]] 
    # initial objective
    objective = cp.square(inferred_parameters[3]-(states[0][2]-states[0][0])) +\
        cp.multiply(inferred_parameters[0], cp.square(states[0][1] - desired_speed))+\
        cp.multiply(inferred_parameters[1], cp.square(controls[0])) 
    for t in range(1, T):
        # objective
        objective += cp.square(inferred_parameters[3]-(states[t][2]-states[t][0])) +\
            cp.multiply(inferred_parameters[0], cp.square(states[t][1] - desired_speed))+\
            cp.multiply(inferred_parameters[1], cp.square(controls[t])) 
        # dynamics constraints
        constraints += [states[t] == A @ states[t-1] +\
            B1 @ controls[t-1] +\
            B2 @ acceleration_of_other_car] 
        # control constraints
        constraints += [cp.norm(controls[t], 'inf') <= control_limit]
        constraints += [states[t][0] <= states[t][2]- inferred_parameters[2]] 
    problem = cp.Problem(cp.Minimize(objective), constraints)
    return CvxpyLayer(problem, variables=[states[0],
                                        states[1],
                                        states[2],
                                        states[3],
                                        states[4],
        controls[0],
                                        controls[1],
                                        controls[2],
                                        controls[3],
                                        controls[4],
                                        ], parameters=[x, 
                                                                    acceleration_of_other_car, 
                                                                    inferred_parameters])

x0 = torch.tensor([0., 0.1, 1.0, 0.1])
acceleration_of_other_car = torch.tensor([0.])
theta_0 = 1.0 # 
theta_1 = 10.0
safe_distance = 0.5
desired_speed = 0.1
comfortable_distance = 0.1

mpc_problem = construct_differentiable_mpc_problem(desired_speed)
acceleration = mpc_problem(x0, acceleration_of_other_car, torch.tensor([theta_0, 
                                                                        theta_1, 
                                                                        safe_distance,
                                                                        comfortable_distance]), 
                        solver_args={"verbose":True, 
                                    "max_iters":10000000, 
                                    "eps_abs":1e-5,
                                    "eps_rel":1e-5,
                                    "eps_infeas":1e-5}
                        )
acceleration

------------------------------------------------------------------
	       SCS v3.2.3 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 45, constraints m: 85
cones: 	  z: primal zero / dual free vars: 20
	  l: linear vars: 20
	  q: soc vars: 45, qsize: 15
settings: eps_abs: 1.0e-05, eps_rel: 1.0e-05, eps_infeas: 1.0e-05
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 10000000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 133, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 2.01e+02  1.00e+01  1.02e+04 -5.11e+03  1.00e-01  4.83e-04 
   125| 2.35e-09  3.34e-09  2.67e-08  4.05e+00  1.72e+00  7.11e-04 
----

(tensor([5.7298e-12, 1.0000e-01, 1.0000e+00, 1.0000e-01]),
 tensor([0.0100, 0.1005, 1.0100, 0.1000]),
 tensor([0.0201, 0.1008, 1.0200, 0.1000]),
 tensor([0.0301, 0.1009, 1.0300, 0.1000]),
 tensor([0.0402, 0.1009, 1.0400, 0.1000]),
 tensor([0.0054]),
 tensor([0.0027]),
 tensor([0.0009]),
 tensor([-8.9143e-06]),
 tensor([1.8327e-16]))

In [6]:


def construct_mpc_problem(x, 
                        inferred_parameters, 
                        desired_speed, 
                        acceleration_of_other_car):
    # belows are the parameters of the MPC problem
    # x = cp.Parameter(n)
    # inferred_parameters = cp.Parameter(3, nonneg=True) # this is the parameters of the problem
    # acceleration_of_other_car = cp.Parameter(1)
    # belows are the decision variables of the MPC problem
    states = [cp.Variable(n) for _ in range(T)]
    controls = [cp.Variable(m) for _ in range(T)]
    # initial constraints
    constraints = [states[0] == x, 
        cp.norm(controls[0], 'inf') <= control_limit,
        states[0][0] <= states[0][2] - inferred_parameters[2]] 
    # initial objective
    objective = cp.square(inferred_parameters[3]-(states[0][2]-states[0][0])) +\
        cp.multiply(inferred_parameters[0], cp.square(states[0][1] - desired_speed))+\
        cp.multiply(inferred_parameters[1], cp.square(controls[0])) 
    for t in range(1, T):
        # objective
        objective += cp.square(inferred_parameters[3]-(states[t][2]-states[t][0])) +\
            cp.multiply(inferred_parameters[0], cp.square(states[t][1] - desired_speed))+\
            cp.multiply(inferred_parameters[1], cp.square(controls[t])) 
        # dynamics constraints
        constraints += [states[t] == A @ states[t-1] +\
            B1 @ controls[t-1] +\
            B2 @ acceleration_of_other_car] 
        # control constraints
        constraints += [cp.norm(controls[t], 'inf') <= control_limit]
        constraints += [states[t][0] <= states[t][2]- inferred_parameters[2]] 
    problem = cp.Problem(cp.Minimize(objective), constraints)
    problem.solve()
    print("sollution status: ", problem.status)
    return [states[0].value,
            states[1].value,
            states[2].value,
            states[3].value,
            states[4].value,
            controls[0].value, 
            controls[1].value,
            controls[2].value,
            controls[3].value,
            controls[4].value], objective.value
    # return CvxpyLayer(problem, variables=[controls[0]], parameters=[x, acceleration_of_other_car, inferred_parameters])


In [7]:
# control_limit = 100.0 # this is the constraint on the control
# T = 5 # look-ahead horizon
# x0 = torch.tensor([8., 1, 10, 1])
# x0 = torch.tensor([0., 100, 1000, 100])
# x = x0
# acceleration_of_other_car = np.array([0.])
# theta_0 = 1.0 # desired speed cost
# theta_1 = 1.0 # control cost
# safe_distance = 2.
# desired_speed = 1.

# theta_0 = 1.0 # 
# theta_1 = 1.0
# safe_distance = 500.
# desired_speed = 1340.

inferred_parameters = np.array([theta_0, theta_1, safe_distance, comfortable_distance])
# acceleration_of_other_car = np.array([0.0])
acceleration, obj = construct_mpc_problem(x0.cpu().numpy(), 
                                        inferred_parameters, 
                                        desired_speed,
                                        acceleration_of_other_car.cpu().numpy())
# acceleration = mpc_problem(x0, acceleration_of_other_car, torch.tensor([theta_0, theta_1, safe_distance]))
acceleration


sollution status:  optimal


[array([8.40223092e-10, 1.00000000e-01, 1.00000000e+00, 1.00000000e-01]),
 array([0.01      , 0.10053678, 1.01      , 0.1       ]),
 array([0.02005368, 0.10080413, 1.02      , 0.1       ]),
 array([0.03013409, 0.10089232, 1.03      , 0.1       ]),
 array([0.04022332, 0.10089143, 1.04      , 0.1       ]),
 array([0.00536776]),
 array([0.00267354]),
 array([0.00088194]),
 array([-8.91432673e-06]),
 array([0.])]

In [4]:
obj

array([8723046.])